- We get weights
-  Windowed Lyapunov exponents
-  Calculate granger causality for le-window vs misclassification
-  Find non causal weights
-  Remove non causal weight to build sparse network
-  Set a threshold for training accuracy, run original and sparse networks till they reach the thresholds

In [1]:
from statsmodels.tsa.stattools import grangercausalitytests

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller

In [672]:
stat = False
init = "2"
data = "Vowel"
le = np.genfromtxt(f"/home/sc/chaos dyn/Results/{data}/LE_window/window_LE_5epochs_{init}.csv",delimiter=',')
acc = np.genfromtxt(f"/home/sc/chaos dyn/Results/{data}/point 1/acc/acc_{init}.csv",delimiter=',')

In [566]:
# le = le[:25,39200:]
# le.shape
le = le.T

In [567]:
window_size = 50
overlap = 0.1
iterates = le.shape[1]

mis_risk = []
for i in range(0,iterates):
    
    r = int(window_size*(1-overlap)*(i+1)+window_size*overlap)
    # r=i
    
    # mis_risk.append(100-acc[i])
    mis_risk.append(1-acc[r-1][0])

In [568]:
if stat:
    count = 0
    for i in range(le.shape[0]):
        if (adfuller(le[i,:])[1]>0.05):
            count+=1   
    print(count)
    if (count>0):
        le = np.diff(le)
    
    if adfuller(mis_risk)[1]>=0.05:
        print("Acc not stationary")
        mis_risk = np.diff(mis_risk)

if data != "mnist":
    le = le.T

In [569]:
mis_risk = np.array(mis_risk)

In [573]:
maxlag = 14
df = pd.DataFrame(le)
pval = []
for c in df:#df.columns:
    # test_result = grangercausalitytests(df[[60,c]], maxlag=20, verbose=False)
    try:
      test_result = grangercausalitytests(np.array((mis_risk,df.iloc[:,c])).T, maxlag=maxlag, verbose=False)
    except:
      pval.append(0)
      continue
#     test_result = grangercausalitytests(np.array((mis_risk,df.iloc[:,c])), maxlag=20, verbose=False)
    p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
    f_values = [round(test_result[i+1][0]['ssr_chi2test'][0],4) for i in range(maxlag)]
    min_p_value = np.min(p_values)
    pval.append(min_p_value)

In [574]:
p = pd.Series(pval)
connections = p.sort_values()[-len(p[p>=0.05]):].index
len(p[p>=0.05])

2

In [575]:
connections

Int64Index([6, 26], dtype='int64')

In [38]:
pd.DataFrame(connections).to_csv(f"/home/sc/chaos dyn/chaotic_pruning/masks/mask_{data}_{init}.csv",header= False ,index = False)